In [1]:
import gym

In [2]:
env=gym.make('CartPole-v0')

In [3]:
env.action_space

Discrete(2)

In [4]:
env.observation_space



Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)

In [5]:
env.observation_space.shape[0]


4

In [6]:
env.reset

<bound method TimeLimit.reset of <TimeLimit<CartPoleEnv<CartPole-v0>>>>

In [7]:
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import deque
import random
import tensorflow as tf
from tensorflow import keras
physical_devices=tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0],True)



In [8]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.memory=deque(maxlen=2000)
        self.gamma=0.95 #discount factor
        self.epsilon=1.0
        self.epsilon_decay=0.995
        self.minn_epsilon=0.01
        self.learning_rate=0.001
        self.model=self.create_model()
        
    def create_model(self):
        model=keras.Sequential()
        model.add(keras.layers.Dense(units=24,input_shape=[self.state_size],activation=tf.nn.relu))
        model.add(keras.layers.Dense(units=24,activation=tf.nn.relu))
        model.add(keras.layers.Dense(units=2,activation=tf.keras.activations.linear))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),loss='mse')
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    def act(self,state):
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state)[0])
    def train(self,batch_size):
        mini_batch=random.sample(self.memory,batch_size)
        for exp in mini_batch:
            state,action,reward,next_state,done=exp
            if not done:
                target=reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target=reward
            target_f=self.model.predict(state)
            target_f[0][action]=target
            self.model.fit(state,target_f,epochs=1,verbose=0)
        if self.epsilon>self.minn_epsilon:
            self.epsilon*=self.epsilon_decay
            
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
        
        
        
        

In [9]:
no_epi=1000
output_dir="model_weights/"

In [13]:
agent=Agent(state_size=4,action_size=2)
done=False

In [14]:
for e in range (no_epi):
    state=env.reset()
    state=np.reshape(state,(1,4))
    sum1=0
    for t in range(5000):
        env.render()
        action=agent.act(state)
        next_state,reward,done,other_info=env.step(action)
        reward=reward if not done else -10
        next_state=np.reshape(next_state,(1,4))
        sum1=sum1+reward
        agent.remember(state,action,reward,next_state,done)
        state=next_state
        if done:
            print("game episode: {}/{} Score: {}".format(e,no_epi,t))
            break
    if len(agent.memory)>32:
        agent.train(32)
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+'.hdf5')

print("deep q learner trained")
env.close()
            
        

game episode: 0/1000 Score: 22
game episode: 1/1000 Score: 15
game episode: 2/1000 Score: 77
game episode: 3/1000 Score: 21
game episode: 4/1000 Score: 37
game episode: 5/1000 Score: 37
game episode: 6/1000 Score: 18
game episode: 7/1000 Score: 15
game episode: 8/1000 Score: 13
game episode: 9/1000 Score: 31
game episode: 10/1000 Score: 16
game episode: 11/1000 Score: 27
game episode: 12/1000 Score: 11
game episode: 13/1000 Score: 17
game episode: 14/1000 Score: 24
game episode: 15/1000 Score: 19
game episode: 16/1000 Score: 12
game episode: 17/1000 Score: 30
game episode: 18/1000 Score: 23
game episode: 19/1000 Score: 24
game episode: 20/1000 Score: 10
game episode: 21/1000 Score: 13
game episode: 22/1000 Score: 12
game episode: 23/1000 Score: 14
game episode: 24/1000 Score: 44
game episode: 25/1000 Score: 11
game episode: 26/1000 Score: 17
game episode: 27/1000 Score: 18
game episode: 28/1000 Score: 20
game episode: 29/1000 Score: 23
game episode: 30/1000 Score: 27
game episode: 31/1

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x000001D804085B80>
Traceback (most recent call last):
  File "C:\Users\pranjal bhatt\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 545, in __del__
    gen_dataset_ops.delete_iterator(
  File "C:\Users\pranjal bhatt\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1262, in delete_iterator
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


game episode: 133/1000 Score: 33
game episode: 134/1000 Score: 19
game episode: 135/1000 Score: 44
game episode: 136/1000 Score: 16
game episode: 137/1000 Score: 58
game episode: 138/1000 Score: 33
game episode: 139/1000 Score: 17
game episode: 140/1000 Score: 11
game episode: 141/1000 Score: 25
game episode: 142/1000 Score: 25
game episode: 143/1000 Score: 47
game episode: 144/1000 Score: 19
game episode: 145/1000 Score: 11


KeyboardInterrupt: 